In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os


import cv2
import shutil
import numpy as np
import pandas as pd
import albumentations as A

from PIL import Image, ImageDraw
#from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir
from utils.data_augmentation import batch_augment_with_bbox, batch_augment_with_mask, batch_invers_FDA

In [37]:
import numpy as np
import cv2 as cv
import time
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
assert img is not None, "file could not be read, check with os.path.exists()"
#img = cv.medianBlur(img,5)
print(img.shape)
cimg = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
img = img.mean(axis=-1)
start = time.time()
img = np.array(img, dtype=np.uint8)

# thresholding
#img = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,5,1)
img = cv.Canny(img, 100, 200)
Image.fromarray(img).show()
#
#circles = cv.HoughCircles(img,cv.HOUGH_GRADIENT,1,5,param1=50,param2=30,minRadius=1,maxRadius=20)
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 5, np.array([]), 40, 23, 1,20)
circles = np.uint16(np.around(circles))
print(time.time()-start)
for i in circles[0,:]:
    # draw the outer circle
    cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

cv.imshow('detected circles',cimg)
cv.waitKey(0)
cv.destroyAllWindows()


(1000, 1000, 3)
3.4560039043426514


In [19]:
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
bbox = np.loadtxt('datasets/CF_simulation/bbox/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.txt', delimiter=' ')
bbox = np.concatenate((bbox, bbox[:,0:1]), axis=-1)[:,1:]
img_h, img_w, _ = img.shape
#print(img.shape)
r = np.random.default_rng()
resize_h, resize_w = r.random()*0.5, r.random()*0.9
resize_h, resize_w = int(resize_h*img_h), int(resize_w*img_w)
print(resize_h, resize_w)
augmentor = A.Compose([
    # Transform (random noises)
    A.Resize(resize_h, resize_w, p=1.0),
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(),
    # Crop
    #A.RandomCrop(width=450, height=450),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=90, p=0.5),
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
    #A.FDA(reference_images=[cv2.imread('datasets/CF_simulation/real_image/test/hdt5_0010.tif')],read_fn=lambda x:x,p=1)
    ]
    ,bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

augmented = augmentor(image=img,bboxes=bbox)
Image.fromarray(augmented['image']).show()

270 333


In [3]:
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
bbox = np.loadtxt('datasets/CF_simulation/bbox/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.txt', delimiter=' ')
bbox = np.concatenate((bbox, bbox[:,0:1]), axis=-1)[:,1:]

In [85]:
temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
target_temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
aug = A.Compose([A.FDA([target_temp], p=1, read_fn=lambda x: x)])
result = aug(image=temp)

In [7]:
augmented = augmentor(image=img,bboxes=bbox)

In [8]:
Image.fromarray(augmented['image']).show()

In [9]:
np.array(augmented['bboxes'])

array([[0.5 , 0.95, 1.  , 0.1 , 0.  ],
       [0.5 , 0.8 , 1.  , 0.1 , 0.  ],
       [0.5 , 0.65, 1.  , 0.1 , 0.  ],
       [0.5 , 0.5 , 1.  , 0.1 , 0.  ],
       [0.5 , 0.35, 1.  , 0.1 , 0.  ],
       [0.5 , 0.2 , 1.  , 0.1 , 0.  ],
       [0.5 , 0.05, 1.  , 0.1 , 0.  ]])

In [119]:
res = np.array(augmented['bboxes'])
res = np.concatenate((res[:,-1:], res), axis=-1)[:,:-1]
res

array([[0.        , 0.79821737, 0.61650781, 0.40356525, 0.76698438],
       [0.        , 0.72620155, 0.5829415 , 0.54155936, 0.834117  ],
       [0.        , 0.58518868, 0.54937519, 0.54155936, 0.90124962],
       [0.        , 0.4441758 , 0.51580888, 0.54155936, 0.96838223],
       [0.        , 0.30316292, 0.47466304, 0.54155936, 0.94932607],
       [0.        , 0.21646486, 0.44109673, 0.43292973, 0.88219346],
       [0.        , 0.14595842, 0.40753042, 0.29191685, 0.81506084]])

In [120]:
#res = pd.DataFrame(res)
#res[:,0] = res[:,0].astype(int)
np.savetxt('test.txt',res, fmt=' '.join(['%i']+['%1.4f']*4))

In [ ]:
augmentor = A.Compose([
    # Transform (random noises)
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.1, fog_coef_upper=0.4),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(p=0.2),
    # Crop
    A.BBoxSafeRandomCrop(p=0.5),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=45, p=0.75)
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

#batch_augment_with_bbox(images_path='datasets/CF_simulation/images/', bbox_path='datasets/CF_simulation/bbox', augmentor=augmentor, random_choice=1.0)


In [ ]:
transforms = [
    # Blur effect
    A.MotionBlur(p=0.2, blur_limit=5),
    A.Defocus(p=0.2, alias_blur=0.3),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=180, p=0.75),
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
    # Transform (random noises),
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.02, fog_coef_upper=0.1),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Crop
    #A.BBoxSafeRandomCrop(p=0.5),
    A.RandomCrop(640, 640, p=0.1),
]

batch_augment_with_mask(images_path='datasets/CF_simulation/images/background', mask_path='datasets/CF_simulation/mask/background', transforms=transforms, FDA_targets=None, random_choice=1000)

In [12]:
Image.open('D:\\xie\\Vision\\mcp_yolo\\datasets\\CF_simulation\\mask\\augmented\\01_line_Radius4_2_Density1500_1000_Width100_Reflection_0.tif').show()
Image.open('D:\\xie\\Vision\\mcp_yolo\\datasets\\CF_simulation\\images\\augmented\\01_line_Radius4_2_Density1500_1000_Width100_Reflection_0.tif').show()

In [16]:
for imp in list_file_r('datasets/CF_simulation/mask/background'):
    img=np.array(Image.open(imp))
    img = Image.fromarray(img>0)
    img.save(imp)

In [59]:
# apply inverseFDA
batch_invers_FDA(src='datasets/CF_simulation/images/augmented/train', FDA_targets='datasets/CF_simulation/images/augmented/val', beta=0.4, p=0.2)

In [31]:
from utils.fft import inverseFDA
import PIL.Image as Image
import numpy as np

src_path = 'datasets/CF_simulation/images/supp2/augmented/01_line_Radius3_1_Density4000_500_Width20_Reflection_02.tif'
tgt_path = 'datasets/CF_simulation/real_image/test_gs/hdt_0146.tif'
tgt_path = 'datasets/CF_simulation/real_image/test_gs/image490.bmp'

src, tgt = Image.open(src_path), Image.open(tgt_path)
tgt = tgt.resize(src.size)
src, tgt = np.array(src), np.array(tgt)
img = Image.fromarray(inverseFDA(src, tgt, beta=0.8))
img.save('0.8.png')

320 320 33 33
